In [1]:
import os

In [2]:
%pwd

'd:\\mlop\\datascience_project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\mlop\\datascience_project'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path 
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [16]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [ ]:
import os
from src.datascience import logger
import urllib.request as request
import zipfile

In [19]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # download the zip fie
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists. Skipping download.")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-06-24 11:49:51,421]: INFO: common: YAML file config\config.yaml read successfully.
[2025-06-24 11:49:51,423]: INFO: common: YAML file params.yaml read successfully.
[2025-06-24 11:49:51,426]: INFO: common: YAML file schema.yaml read successfully.
[2025-06-24 11:49:51,427]: INFO: common: Directory created: artifacts
[2025-06-24 11:49:51,429]: INFO: common: Directory created: artifacts/data_ingestion


[2025-06-24 11:49:52,434]: INFO: 51638424: artifacts/data_ingestion/data.zip downloaded with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4C4C:7EB36:4A51B8:57129A:685A2D5B
Via: 1.1 varnish, 1.1 varnish
Expires: Tue, 24 Jun 2025 04:50:16 GMT
Accept-Ranges: bytes
Date: Tue, 24 Jun 2025 04:49:51 GMT
X-Served-By: cache-sin-wsss1830099-SIN, cache-sin-wsss1830069-SIN
X-Cache: MISS, HIT
X-Cache-Hits: 0, 0
X-Timer: S1750740592.676717,VS0,VE1
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: c2883175c0aafc8929274363cb1bd023b8c08f43
Source-Age: 275




# Data Validation

In [38]:
%pwd

'd:\\mlop\\datascience_project'

In [47]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path 
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [48]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [54]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema= schema
        )
        return data_validation_config

In [ ]:
import os
from src.datascience import logger
import pandas as pd

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}\n")
            
            return validation_status
        except Exception as e:
            raise e

In [58]:
try:
    config= ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-24 21:55:15,289]: INFO: common: YAML file config\config.yaml read successfully.
[2025-06-24 21:55:15,290]: INFO: common: YAML file params.yaml read successfully.
[2025-06-24 21:55:15,292]: INFO: common: YAML file schema.yaml read successfully.
[2025-06-24 21:55:15,292]: INFO: common: Directory created: artifacts
[2025-06-24 21:55:15,298]: INFO: common: Directory created: artifacts/data_validation


# Data Transformation

In [2]:
%pwd

'd:\\mlop\\datascience_project\\research'

In [3]:
import os

os.chdir("../")
%pwd

'd:\\mlop\\datascience_project'

In [4]:

from dataclasses import dataclass 
from pathlib import Path

@dataclass
class DataTransformationConfig: 
    root_dir: Path
    data_path: Path

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories ([config.root_dir]) 
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir, 
            data_path=config.data_path
        )
        return data_transformation_config

In [10]:
import os
from src.datascience import logger
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def split_data(self):
        data = pd.read_csv(self.config.data_path)
        train, test = train_test_split(data, test_size=0.2, random_state=42)
        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info("Data split into train and test sets.")
        logger.info(f"Train set shape: {train.shape}")
        logger.info(f"Test set shape: {test.shape}")

        print(f"Train set shape: {train.shape}")
        print(f"Test set shape: {test.shape}")

In [31]:
with open(Path("artifacts\data_validation\status.txt"), "r") as f:
    status = f.read().split(" ")[-1].strip()

print(status == "True")

True


In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.split_data()
except Exception as e:
    logger.exception(e)
    raise e

[2025-06-25 08:05:43,864]: INFO: common: YAML file config\config.yaml read successfully.
[2025-06-25 08:05:43,876]: INFO: common: YAML file params.yaml read successfully.
[2025-06-25 08:05:43,881]: INFO: common: YAML file schema.yaml read successfully.
[2025-06-25 08:05:43,883]: INFO: common: Directory created: artifacts
[2025-06-25 08:05:43,884]: INFO: common: Directory created: artifacts/data_transformation
[2025-06-25 08:05:43,933]: INFO: 663704289: Data split into train and test sets.
[2025-06-25 08:05:43,941]: INFO: 663704289: Train set shape: (1279, 12)
[2025-06-25 08:05:43,950]: INFO: 663704289: Test set shape: (320, 12)
Train set shape: (1279, 12)
Test set shape: (320, 12)


# Model Trainer

In [56]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path 
    model_name: str 
    alpha: float 
    l1_ratio: float 
    target_column: str

In [57]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [58]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path = config.train_data_path, 
            test_data_path = config.test_data_path, 
            model_name = config.model_name, 
            alpha= params.alpha,
            l1_ratio= params.l1_ratio, 
            target_column= schema.name
        )

        return model_trainer_config

In [59]:
import pandas as pd 
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet 
import joblib

In [60]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig): 
        self.config = config
    
    def train(self): 
        train_data = pd.read_csv(self.config.train_data_path) 
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42) 
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [61]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-06-25 12:09:25,892]: INFO: common: YAML file config\config.yaml read successfully.
[2025-06-25 12:09:25,895]: INFO: common: YAML file params.yaml read successfully.
[2025-06-25 12:09:25,900]: INFO: common: YAML file schema.yaml read successfully.
[2025-06-25 12:09:25,902]: INFO: common: Directory created: artifacts
[2025-06-25 12:09:25,903]: INFO: common: Directory created: artifacts/model_trainer
